In [ ]:
import numpy as np
import math
import grass.script as grass
import uuid

def getcenterlat(rlayer):
    region_center_lat = grass.parse_command('g.region', 
                                        rast=rlayer, 
                                        flags='c', 
                                        delimiter=':')['north-south center']
    rLat = math.radians(float(region_center_lat.split(':')[0]) +
                            (float(region_center_lat.split(':')[1])/60) +
                            (float(region_center_lat.split(':')[2][:-1])/3600))
    return rlat


def meters2degree(lat):
    deg = 111412.84 * math.cos(lat) - 93.5*math.cos(3*lat)
    return deg


def getZFactor(rlayer):
    projinfo=grass.parse_command('g.proj', flags='p', delimiter=':')
    if projinfo['units'] == 'meters':
        zscale = 1
    else:
        zscale = 1/meters2degree(getcenterlat(rlayer))
    return zscale


def join(iterator, delimiter):
    it = map(str, iterator)
    delimiter = str(delimiter)
    string = next(it, '')
    for s in it:
        string += delimiter + s
    return string


def aspectremap(offset):
    remap = np.array([[0, offset, 64],
                      [offset, 45+offset, 128],
                      [45+offset, 90+offset, 1],
                      [90+offset, 135+offset, 2],
                      [135+offset, 180+offset, 4],
                      [180+offset, 225+offset, 8],
                      [225+offset, 270+offset, 16],
                      [270+offset, 315+offset, 32],
                      [315+offset, 360, 64]])
    return remap

def map2rules(remap):
    rules = '\n'.join([join(list(remap[j, :]), delimiter=':') for j in range(0, remap.shape[0])])
    return rules


def writerules(rules, output):
    rulefile = open(output, 'w')
    rulefile.write(rules)
    rulefile.close()


def getcellsize(rlayer):
    res = grass.parse_command('g.region', rast=rlayer, delimiter=':', flags='pm')
    cellsize = (float(res['nsres'])+float(res['ewres']))/2
    return int(cellsize)


def getcellsize(rlayer):
    res = grass.parse_command('g.region', rast=rlayer, delimiter=':', flags='pm')
    cellsize = (float(res['nsres'])+float(res['ewres']))/2
    return int(cellsize)


def angleconversion(rlayer, outRaster, rType='Slope', conversiontype='Radians to Degrees'):
    f = str(uuid.uuid1()).replace('-','')
    p = str(uuid.uuid1()).replace('-','')
    if rType == "Slope":
        grass.run_command('r.mapcalc', expression=f'{f}=90', overwrite=True)
    else:
        grass.run_command('r.mapcalc', expression=f'{f}=180', overwrite=True)
    grass.run_command('r.mapcalc', expression=f'{p}=1.570796', overwrite=True)
    grass.run_command('r.mapcalc', expression=f'{outRaster}={rlayer} * {f} / {p}', overwrite=True) 
    grass.run_command('g.remove', type='raster', name='{f},{p}')



In [ ]:
rlayer='bathy_2015_el'

# slope & aspect
zscale = getZFactor(rlayer)
grass.run_command('r.slope.aspect', 
                  elevation=rlayer, 
                  slope='slope', 
                  aspect='aspect', 
                  zscale=zscale, 
                  overwrite=True)


# classifyAspect
writerules(map2rules(aspectremap(offset=22.5)),'rules')
grass.run_command('r.recode', 
                  input='aspect', 
                  output='newaspect', 
                  rules='rules', 
                  overwrite=True)


# SecondDerivativeSlope
grass.run_command('r.slope.aspect', 
                  elevation='slope', 
                  slope='slopeofslope', 
                  zscale=zscale, 
                  overwrite=True)


# AngleConversion
angleconversion('slope', 'out2')